In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('nyc_parking').getOrCreate()

- Creating spark session

In [3]:
nycdata=spark.read.csv('/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv',header=True)

- Reading the data from common folder

In [4]:
nycdata.columns

['Summons Number',
 'Plate ID',
 'Registration State',
 'Issue Date',
 'Violation Code',
 'Vehicle Body Type',
 'Vehicle Make',
 'Violation Precinct',
 'Issuer Precinct',
 'Violation Time']

In [5]:
nycdata.show(10)

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
|    5092451658| GZH7067|                NY|2016-07-08|             7|             SUBN|       TOYOT|                 0|              0|         0400P|
|    4006265037| FZX9232|                NY|2016-08-23|             5|             SUBN|        FORD|                 0|              0|         0233P|
|    8478629828| 66623ME|                NY|2017-06-14|            47|             REFG|

In [6]:
from pyspark.sql.functions import col,countDistinct,when

In [7]:
from pyspark.sql.functions import from_unixtime,unix_timestamp

In [8]:
from pyspark.sql.functions import to_date,year,month,dayofmonth

In [9]:
nycdata.groupby(year(to_date(col('Issue Date'),'yyyy-MM-dd')).alias('years')).agg({'Issue Date':'count'}).sort('years').show(40)

+-----+-----------------+
|years|count(Issue Date)|
+-----+-----------------+
| 1972|                2|
| 1973|                2|
| 1974|                1|
| 1976|                1|
| 1977|                1|
| 1984|                1|
| 1985|                1|
| 1990|                2|
| 1991|                3|
| 1994|                1|
| 1996|                1|
| 1997|                1|
| 2000|              185|
| 2001|                2|
| 2002|                1|
| 2003|                1|
| 2004|                2|
| 2005|                1|
| 2006|                8|
| 2007|               18|
| 2008|                4|
| 2009|                3|
| 2010|               48|
| 2011|               22|
| 2012|               87|
| 2013|               70|
| 2014|              120|
| 2015|              419|
| 2016|          5368391|
| 2017|          5431918|
| 2018|             1057|
| 2019|              472|
| 2020|               22|
| 2021|               22|
| 2022|                4|
| 2023|     

- We can see from the above table their is wide range years present but we are going to analize the data only for year 2017 as given in the question.
- Filtering data only for that year.

In [10]:
nycdata=nycdata.filter(year(to_date(col('Issue Date'),'yyyy-MM-dd'))==2017)

In [11]:
nycdata.groupby(year(to_date(col('Issue Date'),'yyyy-MM-dd')).alias('years')).agg({'Issue Date':'count'}).sort('years').show()

+-----+-----------------+
|years|count(Issue Date)|
+-----+-----------------+
| 2017|          5431918|
+-----+-----------------+



- Checking if columns are null.

In [12]:
nycdata.filter(col('Summons Number').isNull()).count()

0

In [13]:
nycdata.filter(col('Plate ID').isNull()).count()

0

In [14]:
nycdata.filter(col('Violation Time').isNull()).count()

0

<h3>Examine the data</h3>
1] Find the total number of tickets for the year.

In [15]:
nycdata.agg(countDistinct(col('Summons Number'))).show()

+------------------------------+
|count(DISTINCT Summons Number)|
+------------------------------+
|                       5431918|
+------------------------------+



- Total no of tickets in year 2017 are 5431918.

2] Find out the number of unique states from where the cars that got parking tickets came.

In [16]:
nycdata.groupby('Registration State').agg({'Registration State':'count'}).sort('count(Registration State)',ascending=False).show()

+------------------+-------------------------+
|Registration State|count(Registration State)|
+------------------+-------------------------+
|                NY|                  4273951|
|                NJ|                   475825|
|                PA|                   140286|
|                CT|                    70403|
|                FL|                    69468|
|                IN|                    45525|
|                MA|                    38941|
|                VA|                    34367|
|                MD|                    30213|
|                NC|                    27152|
|                TX|                    18827|
|                IL|                    18666|
|                GA|                    17537|
|                99|                    16055|
|                AZ|                    12379|
|                OH|                    12281|
|                CA|                    12153|
|                ME|                    10806|
|            

- We see 99 as one of the Registration state.

- There is a numeric entry '99' in the column, which should be corrected. Replace it with the state having the maximum entries. Provide the number of unique states again.

In [17]:
nycdataflt=nycdata.withColumn('Registration State',when(col('Registration State')=='99','NY').otherwise(col('Registration State')))

- Removing the state with 99 and replacing it with state with max entries.

In [18]:
nycdataflt.groupby('Registration State').agg({'Registration State':'count'}).sort('count(Registration State)',ascending=False).show()

+------------------+-------------------------+
|Registration State|count(Registration State)|
+------------------+-------------------------+
|                NY|                  4290006|
|                NJ|                   475825|
|                PA|                   140286|
|                CT|                    70403|
|                FL|                    69468|
|                IN|                    45525|
|                MA|                    38941|
|                VA|                    34367|
|                MD|                    30213|
|                NC|                    27152|
|                TX|                    18827|
|                IL|                    18666|
|                GA|                    17537|
|                AZ|                    12379|
|                OH|                    12281|
|                CA|                    12153|
|                ME|                    10806|
|                SC|                    10395|
|            

<h3>Aggregation tasks</h3>

1] How often does each violation code occur? Display the frequency of the top five violation codes.

In [19]:
nycdataflt.groupby('Violation Code').agg({'Violation Code':'count'}).sort('count(Violation Code)',ascending=False).show(5)

+--------------+---------------------+
|Violation Code|count(Violation Code)|
+--------------+---------------------+
|            21|               768087|
|            36|               662765|
|            38|               542079|
|            14|               476664|
|            20|               319646|
+--------------+---------------------+
only showing top 5 rows



2] How often does each 'vehicle body type' get a parking ticket? How about the 'vehicle make'?

In [20]:
nycdataflt.groupby('Vehicle Body Type').agg({'Vehicle Body Type':'count'}).sort('count(Vehicle Body Type)',ascending=False).show(5)

+-----------------+------------------------+
|Vehicle Body Type|count(Vehicle Body Type)|
+-----------------+------------------------+
|             SUBN|                 1883954|
|             4DSD|                 1547312|
|              VAN|                  724029|
|             DELV|                  358984|
|              SDN|                  194197|
+-----------------+------------------------+
only showing top 5 rows



- Above are the vehicle body's which get max tickets.

In [21]:
nycdataflt.groupby('Vehicle Make').agg({'Vehicle Make':'count'}).sort('count(Vehicle Make)',ascending=False).show(5)

+------------+-------------------+
|Vehicle Make|count(Vehicle Make)|
+------------+-------------------+
|        FORD|             636844|
|       TOYOT|             605291|
|       HONDA|             538884|
|       NISSA|             462017|
|       CHEVR|             356032|
+------------+-------------------+
only showing top 5 rows



- Above are the vehicle body with max tickets,Ford vehicle's have topped the list.

- Vehicle make

3.1] 'Violation Precinct' (This is the precinct of the zone where the violation occurred). Using this, can you draw any insights for parking violations in any specific areas of the city?

In [22]:
nycdataflt.groupby('Violation Precinct').agg({'Violation Precinct':'count'}).sort('count(Violation Precinct)',ascending=False).show(6)

+------------------+-------------------------+
|Violation Precinct|count(Violation Precinct)|
+------------------+-------------------------+
|                 0|                   925596|
|                19|                   274445|
|                14|                   203553|
|                 1|                   174702|
|                18|                   169131|
|               114|                   147444|
+------------------+-------------------------+
only showing top 6 rows



- We can see from the above table that area 19 is where the maximum violation of traffic rules happen.

3.2] 'Issuer Precinct' (This is the precinct that issued the ticket.)
Here, you would have noticed that the dataframe has the'Violating Precinct' or 'Issuing Precinct' as '0'.These are erroneous entries. Hence, you need to provide the records for five correct precincts. 

In [23]:
nycdataflt.groupby('Issuer Precinct').agg({'Issuer Precinct':'count'}).sort('count(Issuer Precinct)',ascending=False).show(6)

+---------------+----------------------+
|Issuer Precinct|count(Issuer Precinct)|
+---------------+----------------------+
|              0|               1078406|
|             19|                266961|
|             14|                200495|
|              1|                168740|
|             18|                162994|
|            114|                144054|
+---------------+----------------------+
only showing top 6 rows



4] Find the violation code frequencies for three precincts that have issued the most number of tickets. Do these precinct zones have an exceptionally high frequency of certain violation codes? Are these codes common across precincts? 

Answer : From the above Issuer Precinct table we can see that 3 precincts with most no of tickets are 19,14 and 1.

In [24]:
nycdataflt.createOrReplaceTempView('nyc_data')

In [25]:
spark.sql('select count(`Violation Code`) as count_violation from nyc_data where `Issuer Precinct` =19').show()

+---------------+
|count_violation|
+---------------+
|         266961|
+---------------+



In [26]:
spark.sql('select count(`Violation Code`) as count_violation from nyc_data where `Issuer Precinct` =14').show()

+---------------+
|count_violation|
+---------------+
|         200495|
+---------------+



In [27]:
spark.sql('select count(`Violation Code`) as count_violation from nyc_data where `Issuer Precinct` =1').show()

+---------------+
|count_violation|
+---------------+
|         168740|
+---------------+



- We try to analyze Violation codes for these Precincts.

In [28]:
spark.sql('select `Violation Code`,count(*) as freq_violation from nyc_data where `Issuer Precinct` =19 group by `Violation Code` order by freq_violation desc ').show(10)

+--------------+--------------+
|Violation Code|freq_violation|
+--------------+--------------+
|            46|         48445|
|            38|         36386|
|            37|         36056|
|            14|         29797|
|            21|         28415|
|            20|         14629|
|            40|         11416|
|            16|          9926|
|            71|          7493|
|            19|          6856|
+--------------+--------------+
only showing top 10 rows



In [29]:
spark.sql('select `Violation Code`,count(*) as freq_violation from nyc_data where `Issuer Precinct` =1 group by `Violation Code` order by freq_violation desc ').show(10)

+--------------+--------------+
|Violation Code|freq_violation|
+--------------+--------------+
|            14|         38354|
|            16|         19081|
|            20|         15408|
|            46|         12745|
|            38|          8535|
|            17|          7526|
|            37|          6470|
|            31|          5853|
|            69|          5672|
|            19|          5375|
+--------------+--------------+
only showing top 10 rows



In [30]:
spark.sql('select `Violation Code`,count(*) as freq_violation from nyc_data where `Issuer Precinct` =14 group by `Violation Code` order by freq_violation desc ').show(10)

+--------------+--------------+
|Violation Code|freq_violation|
+--------------+--------------+
|            14|         45036|
|            69|         30464|
|            31|         22555|
|            47|         18364|
|            42|         10027|
|            46|          7679|
|            19|          7031|
|            84|          6743|
|            82|          5052|
|            40|          3582|
+--------------+--------------+
only showing top 10 rows



- From the above tables with top 10 entries of Violation codes we can see that Precincts 19,14,1 have in common 14,19 and 46 Violation code's.

5] Find out the properties of parking violations across different times of the day:

In [31]:
spark.sql('select sum(cast(isNull(`Violation Time`) as int))  from nyc_data').show()

+------------------------------------------+
|sum(CAST((Violation Time IS NULL) AS INT))|
+------------------------------------------+
|                                         0|
+------------------------------------------+



- Checking if Violation code have any null values from the above output we can say no.

In [32]:
from pyspark.sql.functions import split,substring,length,size,array,substring_index,lower

In [33]:
spark.sql('select t1.`Violation Time` from (select `Violation Time`,length(`Violation Time`) as ln  from nyc_data) as t1 where t1.ln!=5').show()

+--------------+
|Violation Time|
+--------------+
|           nan|
|          0557|
|           nan|
|          0855|
|           nan|
|           nan|
|          0515|
|          0316|
|          0651|
|          1037|
|           nan|
|          0446|
|           nan|
|           nan|
|           nan|
|           nan|
|           nan|
|           nan|
|           nan|
|           nan|
+--------------+
only showing top 20 rows



- We can see from above table certain violation time are null and do not come with AM or PM format,So replacing them with most common occuring time.

In [34]:
spark.sql('select count(*) from (select `Violation Time`,length(`Violation Time`) as ln  from nyc_data) as t1 where t1.ln!=5').show()

+--------+
|count(1)|
+--------+
|      23|
+--------+



- We can see the count of such data is small i,e 23 So we replace it with most common occuring time.Without replacing we cannot continue further analysis.

In [35]:
spark.sql('select `Violation Time` as vl,count(*) as cnt from nyc_data group by vl order by cnt desc').show()

+-----+-----+
|   vl|  cnt|
+-----+-----+
|0836A|14492|
|1136A|13808|
|1140A|13546|
|0936A|13128|
|0940A|12763|
|0906A|12641|
|0840A|12497|
|1145A|12235|
|0945A|12193|
|1138A|12077|
|0910A|12041|
|1142A|11928|
|1139A|11880|
|0938A|11843|
|0839A|11762|
|0845A|11748|
|0806A|11721|
|0939A|11706|
|0941A|11656|
|1137A|11644|
+-----+-----+
only showing top 20 rows



- So the most common occuring time is 8:36 am. So we replace with this value.

In [36]:
nycdataflt=nycdataflt.withColumn('Violation Time',when(length(col('Violation Time'))!=5,'0836A').otherwise(col('Violation Time')))

In [37]:
nycdataflt.where(length(col('Violation Time'))!=5).show()

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+



- Since we need to form the buckets of time,We only need the hour of the day not minutes.So we just extract the hours.
- We first convert the 12hr time zone into 24hr time for only hours.

In [38]:
nycdataflt=nycdataflt.withColumn('Timestamp',substring(col('Violation Time'),5,1))

- Extracting the am or pm imfo.

In [39]:
nycdataflt=nycdataflt.withColumn('hourofday',substring(col('Violation Time'),1,2).cast('int'))

- Extracting the hour of day.

- for 24 hour time zone we take the hours from 0 to 23.
- So the am time if from 0hr to 11 and pm time is from 12  to 23.
- if the pm time is 12:36 it is completely valid but not the am.We keep the PM time with 12hr same and add 12 to rest of hours of day.
- for AM we keep all hours same except 12,we take it as 0.

In [40]:
nycdataflt=nycdataflt.withColumn('hourofday',when((lower(col('Timestamp'))=='p') & (col('hourofday')!=12),col('hourofday')+12).otherwise(col('hourofday')))
nycdataflt=nycdataflt.withColumn('hourofday',when((lower(col('Timestamp'))=='a') & (col('hourofday')==12),0).otherwise(col('hourofday')))

In [41]:
nycdataflt.show(5)

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+---------+---------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|Timestamp|hourofday|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+---------+---------+
|    8478629828| 66623ME|                NY|2017-06-14|            47|             REFG|       MITSU|                14|             14|         1120A|        A|       11|
|    5096917368| FZD8593|                NY|2017-06-13|             7|             SUBN|       ME/BE|                 0|              0|         0852P|        P|       20|
|    1407740258| 2513JMG|                NY|2017-01-11|            78|             DELV|       FRUEH|               106|            106|    

- From the above sample we can see that we have successfuly converted hr in 24hr format.

In [42]:
from pyspark.ml.feature import Bucketizer

- We use bucketizer to create 6 bins.

In [43]:
nycdataflt.createOrReplaceTempView('nyc_data')

In [44]:
nycdataflt.columns

['Summons Number',
 'Plate ID',
 'Registration State',
 'Issue Date',
 'Violation Code',
 'Vehicle Body Type',
 'Vehicle Make',
 'Violation Precinct',
 'Issuer Precinct',
 'Violation Time',
 'Timestamp',
 'hourofday']

In [45]:
nycdataflt.where(col('hourofday')>24).show(5)

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+---------+---------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|Timestamp|hourofday|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+---------+---------+
|    1420359022| KXY414F|                NJ|2017-03-07|            46|             DELV|       CHEVR|                40|             40|         6815P|        P|       80|
|    1420596238| FHM8683|                NY|2017-06-02|            21|              SDN|       ME/BE|               109|              0|         8715P|        P|       99|
|    1404833780| HGR8521|                NY|2017-02-23|            38|              SDN|       HYUND|                19|            977|    

- We can still see some of the hours more that 24.This is because Violation time is not valid for these records.
- So we replace these values with most common occuring hour of day.

In [46]:
spark.sql('select hourofday,count(*) as cnt from nyc_data group by hourofday order by cnt desc').show()

+---------+------+
|hourofday|   cnt|
+---------+------+
|        9|595631|
|       11|574632|
|       13|549287|
|       12|510135|
|        8|503869|
|       10|489457|
|       14|466068|
|       15|314468|
|       16|295983|
|        7|270629|
|       17|211173|
|        6|121552|
|       18|104284|
|       21| 55322|
|       20| 49221|
|        1| 46069|
|        0| 45701|
|        5| 43154|
|       22| 42540|
|        2| 40312|
+---------+------+
only showing top 20 rows



- We see that 9th hr is the most occuring one.

In [47]:
spark.sql('select count(*) as cnt from nyc_data where hourofday>=24').show()

+---+
|cnt|
+---+
| 55|
+---+



- Their are only 55 records with such defect,So we can replace with proper data without this we cannot use bucketizer.

In [48]:
nycdataflt=nycdataflt.withColumn('hourofday',when(col('hourofday')>=24,9).otherwise(col('hourofday')))

In [49]:
nycdataflt.createOrReplaceTempView('nyc_data')

In [50]:
spark.sql('select count(*) as cnt from nyc_data where hourofday>=24').show()

+---+
|cnt|
+---+
|  0|
+---+



In [51]:
splits=[0,4,8,12,16,20,24]
bucket=Bucketizer(splits=splits,inputCol='hourofday',outputCol='buckets')

- hours [0-4) put in buckets 0.0.
- hours [4-8) put in buckets 1.0.
- hours [8-12) put in buckets 2.0.
- hours [12-16) put in buckets 3.0.
- hours [16-20) put in buckets 4.0.
- hours [20-24] put in buckets 5.0.

In [52]:
nycdataflt=bucket.transform(nycdataflt)

In [53]:
nycdataflt.createOrReplaceTempView('nyc_data')

In [54]:
spark.sql('select buckets,count(*) as cnt from nyc_data group by buckets order by cnt desc').show()

+-------+-------+
|buckets|    cnt|
+-------+-------+
|    2.0|2163644|
|    3.0|1839958|
|    4.0| 637540|
|    1.0| 449881|
|    5.0| 176360|
|    0.0| 164535|
+-------+-------+



- Finding the 3 most commonly occuring Violation codes.

In [55]:
spark.sql('select `Violation Code`,count(*) as cnt from nyc_data where buckets=0.0 group by `Violation Code` order by cnt desc').show(8)

+--------------+-----+
|Violation Code|  cnt|
+--------------+-----+
|            21|36958|
|            40|25867|
|            78|15528|
|            14|15400|
|            20|12184|
|             7|10168|
|            46| 6648|
|            85| 6351|
+--------------+-----+
only showing top 8 rows



In [56]:
spark.sql('select `Violation Code`,count(*) as cnt from nyc_data where buckets=1.0 group by `Violation Code` order by cnt desc').show(8)

+--------------+-----+
|Violation Code|  cnt|
+--------------+-----+
|            14|74114|
|            40|60652|
|            21|57897|
|            20|43146|
|            71|22116|
|             7|18608|
|            46|16414|
|            19|16312|
+--------------+-----+
only showing top 8 rows



In [57]:
spark.sql('select `Violation Code`,count(*) as cnt from nyc_data where buckets=2.0 group by `Violation Code` order by cnt desc').show(8)

+--------------+------+
|Violation Code|   cnt|
+--------------+------+
|            21|598094|
|            36|348165|
|            38|176571|
|            14|149026|
|            46|109716|
|            71| 95559|
|            20| 90569|
|            40| 71063|
+--------------+------+
only showing top 8 rows



In [58]:
spark.sql('select `Violation Code`,count(*) as cnt from nyc_data where buckets=3.0 group by `Violation Code` order by cnt desc').show(8)

+--------------+------+
|Violation Code|   cnt|
+--------------+------+
|            36|286284|
|            38|240721|
|            37|167026|
|            14|141177|
|            46|124119|
|            20|115729|
|            71|103231|
|            21| 74695|
+--------------+------+
only showing top 8 rows



In [59]:
spark.sql('select `Violation Code`,count(*) as cnt from nyc_data where buckets=4.0 group by `Violation Code` order by cnt desc').show(8)

+--------------+------+
|Violation Code|   cnt|
+--------------+------+
|            38|102855|
|            14| 75902|
|            37| 70345|
|             7| 53626|
|            46| 43155|
|            20| 42583|
|            71| 34710|
|            40| 25440|
+--------------+------+
only showing top 8 rows



In [60]:
spark.sql('select `Violation Code`,count(*) as cnt from nyc_data where buckets=5.0 group by `Violation Code` order by cnt desc').show(8)

+--------------+-----+
|Violation Code|  cnt|
+--------------+-----+
|             7|26293|
|            40|22337|
|            14|21045|
|            38|20347|
|            20|15435|
|            46|12278|
|            78| 9719|
|            19| 8669|
+--------------+-----+
only showing top 8 rows



- From the above 6 tables we see that Violations codes 14,40 and 20 are the once which are appearing in all the tables within top 8 list.

- Finding the common time of the day when these Violations happen.

In [61]:
spark.sql('select buckets,count(*) as cnt from nyc_data where `Violation Code`=14 group by buckets order by cnt desc').show()

+-------+------+
|buckets|   cnt|
+-------+------+
|    2.0|149026|
|    3.0|141177|
|    4.0| 75902|
|    1.0| 74114|
|    5.0| 21045|
|    0.0| 15400|
+-------+------+



In [62]:
spark.sql('select buckets,count(*) as cnt from nyc_data where `Violation Code`=40 group by buckets order by cnt desc').show()

+-------+-----+
|buckets|  cnt|
+-------+-----+
|    3.0|71825|
|    2.0|71063|
|    1.0|60652|
|    0.0|25867|
|    4.0|25440|
|    5.0|22337|
+-------+-----+



In [63]:
spark.sql('select buckets,count(*) as cnt from nyc_data where `Violation Code`=20 group by buckets order by cnt desc').show()

+-------+------+
|buckets|   cnt|
+-------+------+
|    3.0|115729|
|    2.0| 90569|
|    1.0| 43146|
|    4.0| 42583|
|    5.0| 15435|
|    0.0| 12184|
+-------+------+



- From the above 3 tables we can see buckets 2 and 3,ie time 8am-12pm and 12pm-16pm is the time when max violations happen.

6] Seasonality in Data.

- We divide the months based on the seasons,summer,winter,fall and winter.

In [64]:
nycdataflt=nycdataflt.withColumn('monthofissue',month(to_date(col('Issue Date'),'yyyy-MM-dd')))

In [65]:
from pyspark.sql.functions import udf

In [66]:
def getseason(data):
    if data==12 or data==1 or data==2: #Winter
        return 1
    elif data==3 or data==4 or data==5: #Spring
        return 2
    elif data==6 or data==7 or data==8: #Summer
        return 3
    elif data==9 or data==10 or data==11: #Fall
        return 4

In [67]:
from pyspark.sql.types import StringType

In [68]:
seasons=udf(getseason,StringType())

In [69]:
nycdataflt=nycdataflt.withColumn('seasons',seasons(col('monthofissue')))

In [70]:
nycdataflt.createOrReplaceTempView('nyc_data')

6.1] find the frequencies of tickets for each season.

In [71]:
spark.sql('select seasons,count(*) as count from nyc_data group by seasons order by count desc').show()

+-------+-------+
|seasons|  count|
+-------+-------+
|      2|2873383|
|      1|1704690|
|      3| 852866|
|      4|    979|
+-------+-------+



- Spring season has got max no of tickets and fall has least no of tickets.

6.2] find the three most common violations for each of these seasons.

In [72]:
spark.sql('select `Violation Code`,count(*) as count from nyc_data where seasons=1 group by `Violation Code` order by count desc').show(8)

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|238183|
|            36|221268|
|            38|187386|
|            14|142262|
|            20| 97996|
|            37| 97812|
|            46| 95991|
|            40| 86804|
+--------------+------+
only showing top 8 rows



In [73]:
spark.sql('select `Violation Code`,count(*) as count from nyc_data where seasons=2 group by `Violation Code` order by count desc').show(8)

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|402424|
|            36|344834|
|            38|271167|
|            14|256397|
|            46|173440|
|            20|157122|
|            37|151049|
|            40|147408|
+--------------+------+
only showing top 8 rows



In [74]:
spark.sql('select `Violation Code`,count(*) as count from nyc_data where seasons=3 group by `Violation Code` order by count desc').show(8)

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|127352|
|            36| 96663|
|            38| 83518|
|            14| 77911|
|            20| 64474|
|            71| 45199|
|            37| 44284|
|            40| 42856|
+--------------+------+
only showing top 8 rows



- Above 3 table have top 5 violation codes almost same with position change.

In [75]:
spark.sql('select `Violation Code`,count(*) as count from nyc_data where seasons=4 group by `Violation Code` order by count desc').show(8)

+--------------+-----+
|Violation Code|count|
+--------------+-----+
|            46|  231|
|            21|  128|
|            40|  116|
|            14|   94|
|            19|   57|
|            20|   54|
|            98|   50|
|            50|   28|
+--------------+-----+
only showing top 8 rows



- All 4 seasons have 21,40,20 and 14 as the common violation code.

7.1] Find the total occurrences of the three most common violation codes.

Answer : From all the above analysis we can say 14,40 and 20 are the most common occuring Violation codes.

In [76]:
spark.sql('select count(*) as total_occurrences from nyc_data where `Violation Code`=14 or `Violation Code`=40 or`Violation Code`=20').show(5)

+-----------------+
|total_occurrences|
+-----------------+
|          1073494|
+-----------------+



- This is the total occurrences of all 3 common violation codes.

7.2] find the total amount collected for the three violation codes with the maximum tickets. State the code that has the highest total collection.

In [77]:
spark.sql('select `Violation Code`,count(*) as count from nyc_data group by `Violation Code` order by count desc').show(5)

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|768087|
|            36|662765|
|            38|542079|
|            14|476664|
|            20|319646|
+--------------+------+
only showing top 5 rows



- 3 violation codes with max no of tickets are 21,36 and 38.

<a href="http://www1.nyc.gov/site/finance/vehicles/services-violation-codes.page">Violation code Fines.</a>

In [78]:
spark.sql('select count(*)*55 as collection from nyc_data where `Violation Code`=21').show()

+----------+
|collection|
+----------+
|  42244785|
+----------+



- For the code 21 we have fine in area Manhattan 96th St. & below $65 and all other area $45 taking the average comes $55.

In [79]:
spark.sql('select count(*)*50 as collection from nyc_data where `Violation Code`=36').show()

+----------+
|collection|
+----------+
|  33138250|
+----------+



- For the code 36 we have fine in area Manhattan 96th St. & below $50 and All other area $50 taking the average comes $50.

In [80]:
spark.sql('select count(*)*50 as collection from nyc_data where `Violation Code`=38').show()

+----------+
|collection|
+----------+
|  27103950|
+----------+



- For the code 21 we have fine in area Manhattan 96th St. & below $65 and all other area $35 taking the average comes $50.

- From the above tables we can see that 21 has highest collection compared to other 2 codes.

In [81]:
spark.sql('select (42244785+60242200)/1000000 as total_collection_for_3').show()

+----------------------+
|total_collection_for_3|
+----------------------+
|            102.486985|
+----------------------+



- Total collection in millions comes out to be 102.49mills for just 3 violations.

In [82]:
spark.stop()

- From all the above findings we can say that:
- Most of the violations happen during 8 am to 4pm.
- Max no of violations happen in spring season i,e in the months of march,april and may.
- least no of violations happen in Fall season i,e in the months of Sep,Oct and Nov.
- Most common violation codes in the various seasons and different time are 14,40 and 20.
- Violation codes with max tickets are 21,36 and 38.
- Fine collected from violations 21,36 and 38 during 2017 comes out to be 102.49 million.